In [9]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [10]:
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10,
dtype=torch.float).scatter_(0, torch.tensor(y), value = 1))
)

In [11]:
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

using cpu device


In [15]:
# 클래스 정의하기
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class:{y_pred}")

Predicted class:tensor([0])


In [19]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [21]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [22]:
print(f"Before ReLU: {hidden1} \n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2534, -0.1154,  0.0908,  0.0797,  0.7701, -0.0425,  0.1853, -0.2205,
         -0.3314, -0.0038, -0.3265,  0.2070, -0.6051,  0.3161, -0.4740,  0.0418,
          0.2465, -0.1478,  0.3036, -0.0033],
        [-0.3017, -0.1723, -0.0319,  0.3145,  0.5176, -0.2237,  0.6699, -0.4904,
         -0.1518,  0.0499, -0.4714,  0.0597, -0.6619,  0.1165, -0.1611,  0.0871,
          0.3158,  0.0228,  0.3403, -0.0780],
        [-0.5019, -0.2297, -0.1343,  0.3404,  0.5223, -0.0013,  0.2907, -0.3250,
         -0.1096,  0.1718, -0.3747,  0.1127, -0.3372,  0.0771, -0.1112,  0.1683,
          0.6214, -0.0253,  0.0951, -0.0523]], grad_fn=<AddmmBackward0>) 


After ReLU: tensor([[0.0000, 0.0000, 0.0908, 0.0797, 0.7701, 0.0000, 0.1853, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2070, 0.0000, 0.3161, 0.0000, 0.0418, 0.2465, 0.0000,
         0.3036, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3145, 0.5176, 0.0000, 0.6699, 0.0000, 0.0000,
         0.0499, 0.0000, 0.0597, 0.0000, 0.1165, 0.0

In [24]:
seq_moduels = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_moduels(input_image)


In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [27]:
print(f"Model structure: {model} \n\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 




In [29]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | size : {param.size()} | value: {param[:2]} \n\n")

Layer: linear_relu_stack.0.weight | size : torch.Size([512, 784]) | value: tensor([[-3.1580e-02,  1.1885e-02, -1.0302e-02,  ..., -1.2215e-02,
         -1.8615e-02,  1.3588e-02],
        [-5.7306e-03, -3.3118e-05,  2.7634e-02,  ...,  2.0554e-02,
         -2.7580e-03,  3.1209e-02]], grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.0.bias | size : torch.Size([512]) | value: tensor([-0.0243,  0.0026], grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.2.weight | size : torch.Size([512, 512]) | value: tensor([[-0.0143, -0.0269,  0.0282,  ...,  0.0243, -0.0354, -0.0240],
        [-0.0319, -0.0192,  0.0266,  ...,  0.0181, -0.0416, -0.0014]],
       grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.2.bias | size : torch.Size([512]) | value: tensor([ 0.0033, -0.0074], grad_fn=<SliceBackward0>) 


Layer: linear_relu_stack.4.weight | size : torch.Size([10, 512]) | value: tensor([[-0.0355,  0.0007,  0.0166,  ...,  0.0106,  0.0380, -0.0116],
        [-0.0216, -0.0163,  0.0055,  ..., -0.